In [189]:
module JuliaMBD

abstract type AbstractComponent end
abstract type AbstractBlock <: AbstractComponent end
abstract type AbstractCompositeBlock <: AbstractBlock end
abstract type AbstractSimpleBlock <: AbstractBlock end
abstract type AbstractPortBlock <: AbstractSimpleBlock end
abstract type AbstractInPortBlock <: AbstractPortBlock end
abstract type AbstractOutPortBlock <: AbstractPortBlock end
abstract type AbstractParameterPortBlock <: AbstractPortBlock end
abstract type AbstractSignal <: AbstractComponent end
abstract type AbstractLineSignal <: AbstractSignal end
abstract type AbstractSymbolSignal <: AbstractSignal end

const Auto = Any

struct UndefBlock <: AbstractBlock end
struct UndefPort <: AbstractPortBlock end
struct UndefSignal <: AbstractSignal end

abstract type AbstractBlockType end
struct GainBlockType <: AbstractBlockType end

default_value(::Type{Int})::Int = 0
default_value(::Type{Float64})::Float64 = 0.0
default_value(::Type{Any}) = 0

mutable struct SymbolSignal{Tv} <: AbstractSymbolSignal
    name::Symbol
    type::Type{Tv}
    default::Tv
    dest::AbstractBlock

    SymbolSignal(name::Symbol) = new{Auto}(name, Auto, default_value(Auto))
    SymbolSignal(name::Symbol, ::Type{Tv}) where Tv = new{Tv}(name, Tv, default_value(Tv))
end

mutable struct InPort{Tv} <: AbstractInPortBlock
    name::Symbol
    type::Type{Tv}
    parent::AbstractBlock
    in::AbstractSignal
    outs::Dict{Symbol,AbstractSignal}

    InPort(name::Symbol) = new{Auto}(name, Auto, UndefBlock(), UndefSignal(), Dict{Symbol,AbstractSignal}())
    InPort(name::Symbol, ::Type{Tv}) where Tv = new{Tv}(name, Tv, UndefBlock(), UndefSignal(), Dict{Symbol,AbstractSignal}())
end

mutable struct OutPort{Tv} <: AbstractOutPortBlock
    name::Symbol
    type::Type{Tv}
    parent::AbstractBlock
    in::AbstractSignal
    outs::Dict{Symbol,AbstractSignal}

    OutPort(name::Symbol) = new{Auto}(name, Auto, UndefBlock(), UndefSignal(), Dict{Symbol,AbstractSignal}())
    OutPort(name::Symbol, ::Type{Tv}) where Tv = new{Tv}(name, Tv, UndefBlock(), UndefSignal(), Dict{Symbol,AbstractSignal}())
end

mutable struct ParameterPort{Tv} <: AbstractParameterPortBlock
    name::Symbol
    type::Type{Tv}
    parent::AbstractBlock
    in::AbstractSignal
    outs::Dict{Symbol,AbstractSignal}

    ParameterPort(name::Symbol) = new{Auto}(name, Auto, UndefBlock(), UndefSignal(), Dict{Symbol,AbstractSignal}())
    ParameterPort(name::Symbol, ::Type{Tv}) where Tv = new{Tv}(name, Tv, UndefBlock(), UndefSignal(), Dict{Symbol,AbstractSignal}())
end

mutable struct LineSignal <: AbstractLineSignal
    name::Symbol
    desc::String
    src::AbstractBlock
    dest::AbstractBlock

    function LineSignal(src::AbstractBlock, dest::AbstractBlock, desc::String)
        line = new(gensym(), desc, src, dest)
        src.outs[line.name] = line
        dest.in = line
        line
    end
end

function connect()

mutable struct SimpleBlock <: AbstractSimpleBlock
    name::Symbol
    desc::String
    inports::Dict{Symbol,AbstractInPortBlock}
    outports::Dict{Symbol,AbstractOutPortBlock}
    parameters::Dict{Symbol,AbstractParameterPortBlock}
    # type::AbstractBlockType

    function SimpleBlock(name::Symbol)
        b = new(name,
            "",
            Dict{Symbol,AbstractInPortBlock}(),
            Dict{Symbol,AbstractOutPortBlock}(),
            Dict{Symbol,AbstractParameterPortBlock}())
        b
    end
end

function set!(b::AbstractSimpleBlock, s::Symbol, x::AbstractInPortBlock)
    l = LineSignal(x, b, "inner")
    x.outs[l.name] = l
    b.inports[s] = x
end

function set!(b::AbstractSimpleBlock, s::Symbol, x::AbstractOutPortBlock)
    l = LineSignal(b, x, "inner")
    x.ins = l
    b.outports[s] = x
end

function set!(b::AbstractSimpleBlock, s::Symbol, x::AbstractParameterPortBlock)
    l = LineSignal(x, b, "inner")
    x.outs[l.name] = l
    b.parameters[s] = x
end

mutable struct SubSystemBlock <: AbstractCompositeBlock
    name::Symbol
    desc::String
    inports::Dict{Symbol,AbstractInPortBlock}
    outports::Dict{Symbol,AbstractOutPortBlock}
    parameters::Dict{Symbol,AbstractParameterPortBlock}
    blocks::Dict{Symbol,AbstractBlock}
    # type::AbstractBlockType

    function SubSystemBlock(name::Symbol)
        b = new(name, "",
            Dict{Symbol,AbstractInPortBlock}(),
            Dict{Symbol,AbstractOutPortBlock}(),
            Dict{Symbol,AbstractParameterPortBlock}(),
            Dict{Symbol,AbstractBlock}())
        b
    end
end

prev(x::AbstractSimpleBlock) = values(x.inports)
next(x::AbstractSimpleBlock) = values(x.outports)

prev(x::UndefBlock) = []
next(x::UndefBlock) = []

prev(x::AbstractSymbolSignal) = []
next(x::AbstractSymbolSignal) = [x.dest]

prev(x::AbstractLineSignal) = [x.src]
next(x::AbstractLineSignal) = [x.dest]

prev(x::UndefPort) = []
next(x::UndefPort) = []

prev(x::UndefSignal) = []
next(x::UndefSignal) = []

prev(x::AbstractInPortBlock) = [x.in]
next(x::AbstractInPortBlock) = values(x.outs)

prev(x::AbstractOutPortBlock) = [x.in]
next(x::AbstractOutPortBlock) = values(x.outs)

prev(x::AbstractParameterPortBlock) = [x.in]
next(x::AbstractParameterPortBlock) = values(x.outs)

"""
all blocks

Get all blocks from a block
"""
function allcomponents(blk::AbstractComponent)
    visited = Set{AbstractComponent}()
    _allblocks(blk, visited)
    collect(visited)
end

function _allblocks(blk::AbstractComponent, visited::Set{AbstractComponent})::Nothing
    if blk in visited
        return nothing
    end
    push!(visited, blk)
    for b = next(blk)
        _allblocks(b, visited)
    end
    for b = prev(blk)
        _allblocks(b, visited)
    end
    nothing
end

"""
tsort

Tomprogical sort
"""
function tsort(bs::Vector{AbstractComponent})
    sorted = AbstractComponent[]
    check = Dict{AbstractComponent,Int}()
    for n = bs
        check[n] = 0 # no visited
    end
    for n = bs
        if check[n] != 2
            _visit(n, check, sorted)
        end
    end
    sorted
end

function _visit(n::AbstractComponent, check::Dict{AbstractComponent,Int}, sorted::Vector{AbstractComponent})
    if check[n] == 1 # under searching
        throw(ErrorException("DAG has a closed path"))
    elseif check[n] == 0 # no visited
        check[n] = 1 # start searching
        for m = next(n)
            _visit(m, check, sorted)
        end
        check[n] = 2 # visited
        pushfirst!(sorted, n)
    end
end

# function _expr!(exprs::Vector{Expr}b::Block)
#     visited = Set()
#     for (s,p) = inports
#         if !(p in visited)
#             to_expr(p)
#         end
#     end
# end

end

Main.JuliaMBD

In [186]:

# JuliaMBD.LineSignal(pout, pin, "s(t)")

In [190]:
b = JuliaMBD.SimpleBlock(:test)
pout = JuliaMBD.OutPort(:a, Float64)
pin = JuliaMBD.InPort(:b, Float64)
JuliaMBD.set!(b, pout.name, pout)
JuliaMBD.set!(b, pin.name, pin)
pext = JuliaMBD.OutPort(:ex)
JuliaMBD.LineSignal(pext, pin, "s(t)")

ErrorException: type SimpleBlock has no field outs

In [188]:
JuliaMBD.tsort(JuliaMBD.allcomponents(b))

6-element Vector{Main.JuliaMBD.AbstractComponent}:
 Main.JuliaMBD.Block(:test, "", Dict{Symbol, Main.JuliaMBD.AbstractInPortBlock}(:b => Main.JuliaMBD.InPort{Float64}(:b, Float64, Main.JuliaMBD.Block(#= circular reference @-3 =#), Main.JuliaMBD.LineSignal(Symbol("##335"), "s(t)", Main.JuliaMBD.OutPort{Any}(:ex, Any, Main.JuliaMBD.UndefBlock(), Main.JuliaMBD.UndefSignal(), Dict{Symbol, Main.JuliaMBD.AbstractSignal}(Symbol("##335") => Main.JuliaMBD.LineSignal(#= circular reference @-3 =#))), Main.JuliaMBD.InPort{Float64}(#= circular reference @-2 =#)), Dict{Symbol, Main.JuliaMBD.AbstractSignal}())), Dict{Symbol, Main.JuliaMBD.AbstractOutPortBlock}(:a => Main.JuliaMBD.OutPort{Float64}(:a, Float64, Main.JuliaMBD.Block(#= circular reference @-3 =#), Main.JuliaMBD.UndefSignal(), Dict{Symbol, Main.JuliaMBD.AbstractSignal}())), Dict{Symbol, Main.JuliaMBD.AbstractParameterPortBlock}(), Dict{Symbol, Main.JuliaMBD.AbstractBlock}())
 Main.JuliaMBD.UndefSignal()
 Main.JuliaMBD.OutPort{Any}(:ex, Any

In [ ]:
"""
all blocks

Get all blocks from a block
"""
function allblocks(blk::AbstractComponent)
    visited = Set{AbstractComponent}()
    _allblocks(blk, visited)
    collect(visited)
end

function _allblocks(blk::AbstractComponent, visited::Set{AbstractComponent})::Nothing
    if blk in visited
        return nothing
    end
    push!(visited, blk)
    for b = next(blk)
        _allblocks(b, visited)
    end
    for b = prev(blk)
        _allblocks(b, visited)
    end
    nothing
end

"""
tsort

Tomprogical sort
"""
function tsort(bs::Vector{AbstractComponent})
    sorted = AbstractComponent[]
    check = Dict{AbstractComponent,Int}()
    for n = bs
        check[n] = 0 # no visited
    end
    for n = bs
        if check[n] != 2
            _visit(n, check, sorted)
        end
    end
    sorted
end

function _visit(n::AbstractComponent, check::Dict{AbstractComponent,Int}, sorted::Vector{AbstractComponent})
    if check[n] == 1 # under searching
        throw(ErrorException("DAG has a closed path"))
    elseif check[n] == 0 # no visited
        check[n] = 1 # start searching
        for m = next(n)
            _visit(m, check, sorted)
        end
        check[n] = 2 # visited
        pushfirst!(sorted, n)
    end
end

###

# function tsort2(blks::Vector{AbstractBasicBlock}, parent::AbstractBlock)
#     l = AbstractBasicBlock[]
#     s = Stack{AbstractBlock}()
#     push!(s, parent)
#     check = Dict{Tuple{AbstractBasicBlock,AbstractBlock},Int}()
#     for n = blks
#         check[(n,first(s))] = 0
#     end
#     for n = blks
#         if check[(n,first(s))] != 2
#             _visit2(n, check, l, s)
#         end
#     end
#     l
# end

# function _visit2(n::AbstractBasicBlock, check, l, s)
#     if !haskey(check, (n,first(s)))
#         check[(n,first(s))] = 0
#     end
#     if check[(n,first(s))] == 1
#         throw(ErrorException("DAG has a closed path"))
#     end
#     if check[(n,first(s))] == 0
#         check[(n,first(s))] = 1
#         for m = nextblk(n)
#             _visit2(m, check, l, s)
#         end
#         check[(n,first(s))] = 2
#         pushfirst!(l, n)
#     end
# end

# function _visit2(n::AbstractInPort, check, l, s)
#     m = get_parent(n)
#     push!(s, m)
#     _visit2(m.definiton.ports[get_name(n)], check, l, s)
#     pop!(s)
# end

# function _visit2(n::Outport, check, l, s)
#     if typeof(first(s)) != GlobalSystem
#     m = get_parent(n)
#     push!(s, m)
#     _visit2(m.definiton.ports[get_name(n)], check, l, s)
#     pop!(s)
# end

